In [15]:
import torch
import datasets
import os
import time
import numpy as np
import sys
import pickle
import torch.nn.functional as F
from transformers import (
    LlamaForCausalLM,
    GenerationConfig
)

### Skip this

In [2]:
llama_causal = LlamaForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
from hqq.models.hf.base import AutoHQQHFModel
from hqq.core.quantize import BaseQuantizeConfig
quant_config = BaseQuantizeConfig(nbits=4, group_size=64)
AutoHQQHFModel.quantize_model(llama_causal, quant_config=quant_config, 
                                    compute_dtype=torch.float16, 
                                    device="cuda")
# 1.4gb

100%|██████████| 155/155 [00:17<00:00,  9.00it/s]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): HQQLinear(in_features=2048, out_features=2048, bias=False)
          (k_proj): HQQLinear(in_features=2048, out_features=256, bias=False)
          (v_proj): HQQLinear(in_features=2048, out_features=256, bias=False)
          (o_proj): HQQLinear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): HQQLinear(in_features=2048, out_features=5632, bias=False)
          (up_proj): HQQLinear(in_features=2048, out_features=5632, bias=False)
          (down_proj): HQQLinear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()


In [7]:
from hqq.models.hf.base import AutoHQQHFModel
AutoHQQHFModel.save_quantized(llama_causal, "./tinyllama_hqq_4_64")

### Continue

In [2]:
from hqq.models.hf.base import AutoHQQHFModel
llama_causal = AutoHQQHFModel.from_quantized("./tinyllama_hqq_4_64")

100%|██████████| 155/155 [00:00<00:00, 11347.63it/s]


In [4]:
from transformers import AutoTokenizer
llama_tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

tensor([[  529, 29989,  1792, 29989, 29958,    13, 10994, 29892,   920,   526,
           366, 29973,     2, 29871,    13, 29966, 29989,   465, 22137, 29989,
         29958,    13, 29902, 29915, 29885,  2599,  2107, 29889,  1128,   508,
           306,  1371,   366,  9826, 29973,     2, 29871,    13, 29966, 29989,
          1792, 29989, 29958,    13, 29902, 29915, 29881,   763,   304,  1510,
          1283,   920, 13563,  1350,   572,  1218,  1736, 29991,     2, 29871,
            13]])

In [57]:
input_ids = llama_tokenizer.apply_chat_template([
    {
        "role": "system",
        "content": "You are now an expert on recognizing emotions in conversations. Respond to any input messages with a classification of the message from the following labels: [angry, happy, other].",
    },
    {
        "role": "user",
        "content": "waaaaaa I dropped my ice cream :(((( this is the WORST DAY EVER!",
    },
], return_tensors='pt')

In [58]:
gen_cfg = GenerationConfig.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
gen_cfg.max_length = 256

In [59]:
llama_tokenizer.decode(llama_causal.generate(input_ids.cuda().long(), generation_config=gen_cfg)[0])

'<|system|>\nYou are now an expert on recognizing emotions in conversations. Respond to any input messages with a classification of the message from the following labels: [angry, happy, other].</s> \n<|user|>\nwaaaaaa I dropped my ice cream :(((( this is the WORST DAY EVER!</s> \n<|assistant|>\nI am not able to experience emotions, but I can provide you with a response to your question.\n\nin this case, you dropped your ice cream, which is a common occurrence. The label "angry" would be appropriate for a message that describes the emotions you experienced when you dropped the ice cream. The label "happy" would be appropriate for a message that describes the emotions you experienced when you saw the ice cream on the ground.\n\nin this case, you could also use the label "other" to describe the emotions you experienced when you realized that you dropped your ice cream. This label would be appropriate for a message that describes the emotions you experienced when you realized that you drop